# Parallel Ensemble Simulations

Docs: <https://diffeq.sciml.ai/stable/features/ensemble/>

## Solving an ODE With Different Initial Conditions

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
# Linear ODE which starts at 0.5 and solves from t=0.0 to t=1.0
prob = ODEProblem((u,p,t)->1.01u, 0.5, (0.0, 1.0))

In [ ]:
# Define a new problem for each trajectory
# Change the initial condition (u0) in this example
function prob_func(prob, i, repeat)
    remake(prob, u0=rand() * prob.u0)
end

You could also obtain the necessary data from outside of the `prob_func()`

```julia
initial_conditions = range(0, stop=1, length=100)

function prob_func(prob, i, repeat)
  remake(prob, u0=initial_conditions[i])
end
```

In [ ]:
ensemble_prob = EnsembleProblem(prob; prob_func=prob_func)

# Ensemble simulations use multithreading by default
sim = solve(ensemble_prob, trajectories=100)

In [ ]:
# Each element of sim is an ODE solution
sim[1]

In [ ]:
plot(sim, linealpha=0.4)

## Solving an SDE with Different Parameters

In [ ]:
function lotka_volterra!(du, u, p, t)
    du[1] = p[1] * u[1] - p[2] * u[1] * u[2]
    du[2] = -3 * u[2] + u[1] * u[2]
end

function g!(du, u, p, t)
    du[1] = p[3] * u[1]
    du[2] = p[4] * u[2]
end

In [ ]:
p = [1.5, 1.0, 0.1, 0.1]
prob = SDEProblem(lotka_volterra!, g!, [1.0, 1.0], (0.0, 10.0), p)

In [ ]:
function prob_func(prob, i, repeat)
    x = 0.3 * rand(2)
    remake(prob, p=[p[1:2];x])
end

In [ ]:
ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, SRIW1(), trajectories=10)

In [ ]:
plot(sim, linealpha=0.6, color=:blue, vars=(0,1))
plot!(sim, linealpha=0.6, color=:red, vars=(0,2))

In [ ]:
summ = EnsembleSummary(sim, 0:0.1:10)
plot(summ, fillalpha=0.5)